<a href="https://colab.research.google.com/github/inspektral/asmc-genre-classification/blob/main/task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split

In [2]:
features_3_sec = pd.read_csv('data/features_3_sec.csv')
features_30_sec = pd.read_csv('data/features_30_sec.csv')


In [3]:
USED_SET = features_3_sec
USED_SET = USED_SET.drop(['filename', 'length'], axis=1)

train, test = train_test_split(USED_SET, test_size=0.2)

In [4]:
# Normalize the data
from sklearn.preprocessing import StandardScaler
def normalize_feature_set(features):
    scaler = StandardScaler()

    scaler.fit(features.drop('label', axis=1))
    scaled_features = scaler.transform(features.drop('label', axis=1))

    scaled_df = pd.DataFrame(scaled_features, columns=features.columns[:-1])
    scaled_df['label'] = features['label']
    # scaled_df = scaled_df.fillna(0)

    return scaled_df


test = normalize_feature_set(test)
train = normalize_feature_set(train)

display(train.head())
display(test.head())


,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,0.964544,-0.193872,-0.235826,-0.555511,0.388493,1.509753,0.990289,1.437028,0.535801,2.261015,...,0.027483,0.622718,0.015741,0.428428,-0.118859,0.358951,-0.575489,0.305839,-0.299566,blues
1,-0.542457,2.168159,0.525268,0.805102,-1.289470,-0.174182,-0.248127,1.291315,-1.265516,1.656009,...,-0.268099,0.743692,-0.328184,0.241637,0.032424,0.422108,-0.468132,0.427740,-0.586535,blues
2,-0.057089,-1.314458,-0.250573,-0.698136,-1.550815,-0.932470,-1.424312,-1.112545,-1.754275,-1.060672,...,-1.033225,-0.867955,-0.938621,0.795569,-0.950180,-1.171314,-0.898485,-0.238769,-0.450801,blues
3,-0.867363,1.390500,-0.268774,-0.281921,0.207664,0.088078,1.194465,1.563408,0.564728,1.644063,...,-0.131021,1.537229,0.106180,0.230627,-0.197360,-0.651113,0.653574,-2.355974,0.996316,blues
4,-0.517508,-0.262062,-0.347807,-0.228296,-1.070329,-0.645566,-1.155973,-0.438520,-1.175945,-0.905076,...,-0.190249,0.949033,0.509693,0.362786,-0.228929,0.974954,-0.025860,1.201646,0.079268,NaN


,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,0.655195,0.249749,1.910041,2.246388,-0.732414,0.139771,0.345265,1.635377,-0.561957,1.507608,...,-0.111391,-0.416873,0.206848,-0.857475,-0.654156,-1.073361,-0.632676,-0.975293,-0.198820,NaN
1,0.107129,-0.367553,0.425173,-0.503757,0.237714,-0.419768,-0.130539,-0.526603,0.105678,-0.312296,...,-0.874125,-1.143350,-0.419814,-0.898297,-0.380200,-0.397057,-0.207590,-0.633199,0.195159,NaN
2,-1.742125,-0.609654,-1.053657,-0.659035,-0.705970,-0.774713,-0.683001,-0.615776,-0.696135,-0.698494,...,0.563845,0.265661,0.036468,-0.349061,-0.162795,-0.774953,0.807217,0.411861,0.227316,NaN
3,-0.150782,-0.364760,-0.726682,-0.480064,0.705355,-0.111976,1.102558,0.084545,0.823598,-0.010282,...,0.075832,-0.393409,-0.195913,-2.036128,-0.038645,-0.378765,-0.287493,0.427439,-0.264923,NaN
4,0.689630,-1.424551,-0.369046,-0.447228,0.665193,-0.157970,0.567166,-0.062760,0.515467,-0.082872,...,-0.781041,-0.129183,-0.929351,-0.607496,-0.860594,-0.624394,-0.791861,-0.940480,-0.840794,blues


In [5]:
# MLP Classifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

X_train = train.drop('label', axis=1)
X_test = test.drop('label', axis=1)

y_train = train['label']
y_test = test['label']

# Encode the labels
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
y_train = labelencoder.fit_transform(y_train)
y_test = labelencoder.fit_transform(y_test)

mlp = MLPClassifier(hidden_layer_sizes=(100, 100, 100), max_iter=1000)
mlp.fit(X_train, y_train)

# Encode the labels
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
y_train = labelencoder.fit_transform(y_train)
y_pred = mlp.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.11261261261261261


In [6]:
# kNN classifier

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

knn = KNeighborsClassifier(n_neighbors=10)

knn.fit(X_train, y_train)

predictions = knn.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
print(accuracy)

0.13013013013013014


In [7]:
# SVM classifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

svm = SVC(kernel='linear')
svm.fit(X_train, y_train)

predictions = svm.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
print(accuracy)

0.0


In [8]:
# Random Forest classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train, y_train)

predictions = rf.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
print(accuracy)


0.052552552552552555


In [9]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)

predictions = lr.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
print(accuracy)


0.010510510510510511
